In [1]:
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import arviz as az

from collections import namedtuple
from dataclasses import dataclass
from scipy.stats import bernoulli, expon

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
RANDOM_SEED = 4000
rng = np.random.default_rng(RANDOM_SEED)

plt.style.use("bmh")

plotting_defaults = dict(
    bins=50,
    kind="hist",
    textsize=10,
)

# What are A/B tests?

A/B testing (also known as split testing) is a process of showing two variants of the same web page to different segments of website visitors at the same time and comparing which variant drives more conversions. 

Specifically, A/B tests are often used in the software industry to determine whether a new feature or changes to an existing feature should be released to users, and the impact of the change on core product metrics (“conversions”). Furthermore:

We can test more than two variants at the same time. We’ll be dealing with how to analyse these tests in this notebook as well.
Exactly what “conversions” means can vary between tests, but two classes of conversions we’ll focus on are:
1. Bernoulli conversions - a flag for whether the visitor did the target action or not (e.g. completed at least one purchase).
2. Value conversions - a real value per visitor (e.g. the dollar revenue, which could also be 0).

# Bernoulli conversions

In [3]:
@dataclass
class BetaPrior:
    alpha: float
    beta: float


@dataclass
class BinomialData:
    trials: int
    successes: int

In [119]:
class ConversionModelTwoVariant:
    def __init__(self, priors: BetaPrior):
        self.priors = priors
    def create_model(self, data):
        trials = [d.trials for d in data]
        successes = [d.successes for d in data]

        with pm.Model() as model:
            prior = pm.Beta("prior",
                            alpha=self.priors.alpha,
                            beta=self.priors.beta,
                            shape=2)
            likelihood = pm.Binomial("likelihood",
                                     n=trials,
                                     p=prior,
                                     shape=2,
                                     observed=successes)
            rel_uplift = pm.Deterministic("rel_uplift_b", prior[1] / prior[0] - 1)

        return model

## Data generation

We generate two datasets:
1. Where the "true" conversion rate in *the same*
2. Where the variant *B* has a higher true converstion rate.

In [120]:
def generate_binomial_data(variants,
                         true_rates,
                         samples_per_variant: int = 100000):
    data = {}
    for variant, rate in zip(variants, true_rates):
        data[variant] = bernoulli.rvs(rate, size=samples_per_variant)
        agg = (pd.DataFrame(data)
                .aggregate(["count", "sum"])
                .rename(index={"count":"trials",
                               "sum": "successes"}))
    return agg

In [121]:
variants = ["A", "B"]
generated = generate_binomial_data(variants=["A", "B"],
                     true_rates=[0.23, 0.23])
print(generated)

data =[BinomialData(**generated[v].to_dict()) for v in variants ]
data

                A       B
trials     100000  100000
successes   23024   22994


[BinomialData(trials=100000, successes=23024),
 BinomialData(trials=100000, successes=22994)]

We create a function to wrap the data generation, sampling, and posterior plots so that we can easily compare the results of both models (strong and weak prior) and under both scenarios (same true rate vs different true rates).

In [122]:
def run_scenario_two_variant(variants,
                             true_rates,
                             samples_per_variant: int,
                             weak_prior: BetaPrior,
                             strong_prior: BetaPrior):
    generated_data = generate_binomial_data(variants,
                                            true_rates,
                                            samples_per_variant)
    data = [
        BinomialData(**generated_data[v].to_dict()) for v in variants
    ]
    with ConversionModelTwoVariant(priors=weak_prior).create_model(data):
        trace_weak = pm.sample(draws=5000)

    with ConversionModelTwoVariant(priors=strong_prior).create_model(data):
        trace_strong = pm.sample(draws=5000)
    
    true_rel_uplift = true_rates[1] / true_rates[0] - 1

    fig, axes =plt.subplots(2, 1, figsize=(7, 7), sharex=True)
    az.plot_posterior(
        trace_weak.posterior["rel_uplift_b"],
        ax=axes[0],
        **plotting_defaults
    )
    axes[0].set_tile(f"True Relative Uplift = {true_rel_uplift:.1%}, {weak_prior}")

    az.plot_posterior(
        trace_strong.posterior["rel_uplift_b"],
        ax=axes[1],
        **plotting_defaults
    )
    axes[1].set_tile(f"True Relative Uplift = {true_rel_uplift:.1%}, {strong_prior}")

    fig.suptitle("B vs. A Relative Uplift")

In [123]:
from dataclasses import dataclass
from typing import Dict, List, Union

In [124]:
@dataclass
class BetaPrior:
    alpha: float
    beta: float

@dataclass
class BinomialData:
    trials: int
    successes: int
    

In [125]:
class ConversionModelTwoVariant:
    def __init__(self, priors: BetaPrior):
        self.priors = priors

    def create_model(self, data: List[BinomialData]) -> pm.Model:
        trials = [d.trials for d in data]
        successes = [d.successes for d in data]
        with pm.Model() as model:
            p = pm.Beta("p", alpha=self.priors.alpha, beta=self.priors.beta, shape=2)
            obs = pm.Binomial("y", n=trials, p=p, shape=2, observed=successes)
            reluplift = pm.Deterministic("reluplift_b", p[1] / p[0] - 1)
        return model

In [ ]:
trace_weak, trace_trong = run_scenario_two_variant(
    variants=["A", "B"],
    true_rates=[0.23, 0.23],
    samples_per_variant=100,
    weak_prior=BetaPrior(alpha=100, beta=100),
    strong_prior=BetaPrior(alpha=10000, beta=10000)
)